In [14]:
import numpy as np
import pandas as pd

M = 100  # кількість суб'єктів
Tmax = 10  # максимальний час спостереження
states = ['CN', 'MCI', 'AD', 'Death']

# Перехідні ймовірності
transition_probs = {
    'CN': [0.6, 0.3, 0.1, 0.0],
    'MCI': [0.1, 0.5, 0.3, 0.1],
    'AD': [0.0, 0.1, 0.6, 0.3],
    'Death': [0.0, 0.0, 0.0, 1.0]
}

# Базові інтенсивності переходу
lambda_0 = {
    'CN': 0.5,
    'MCI': 0.3,
    'AD': 0.2
}

# Функція для генерації наступного стану
def sample_next_state(current):
    return np.random.choice(states, p=transition_probs[current])

# Функція для генерації часу перебування
def generate_sojourn_time(current_state, Z):
    lam = lambda_0.get(current_state, 1.0)
    rate = lam * np.exp(Z)  # експоненційна інтенсивність
    u = np.random.uniform(0, 1)
    return -np.log(1 - u) / rate

# Основний алгоритм
records = []
for m in range(M):
    t = 0
    n = 0
    state = 'CN'
    Z_m = { (i,j): np.random.normal(0, 1) for i in states for j in states if i != 'Death' and j != i }
    
    while t < Tmax and state != 'Death':
        current_state = state
        Z = Z_m.get((current_state, sample_next_state(current_state)), 0)
        W = generate_sojourn_time(current_state, Z)
        
        if t + W > Tmax:
            W = Tmax - t
            censored = 0
            next_state = current_state  # залишаємося в поточному стані через цензурування
        else:
            censored = 1
            next_state = sample_next_state(current_state)
        
        records.append({
            'Subject': m,
            'Start_Time': round(t, 6),
            'From': current_state,
            'To': next_state,
            'Sojourn_Time': round(W, 6),
            'Censored': censored
        })
        t += W
        state = next_state
        n += 1

# Перетворення в DataFrame
sim_data = pd.DataFrame(records)
print(sim_data.head())

   Subject  Start_Time From   To  Sojourn_Time  Censored
0        0    0.000000   CN   CN      4.685867         1
1        0    4.685867   CN   CN      5.314133         0
2        1    0.000000   CN   CN      0.371870         1
3        1    0.371870   CN  MCI      8.096251         1
4        1    8.468121  MCI  MCI      0.106495         1
